# Tableau uploads

## Imports

In [1]:
import pandas as pd
import numpy as np
import wrangle as w
import explore as e
pd.set_option('display.max_columns', None)

In [2]:
df = w.clean_311(w.get_311_data())

In [3]:
df.head()

,open_date,due_date,closed_date,is_late,dept,call_reason,case_type,case_status,source_id,address,council_district,longitude,latitude,days_open,resolution_days_due,days_before_or_after_due,pct_time_of_used,level_of_delay,district_1,district_2,district_3,district_4,district_5,district_6,district_7,district_8,district_9,district_10,voter_turnout_2019,num_of_registered_voters,zipcode,open_month,open_year,open_week,per_capita_income,square_miles
551,2017-01-27,2017-02-03,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"2407 WYOMING ST, San Antonio, 78203",2,2142962.0,13699885.0,1393.0,8,-1385.0,174.125,Very Late Response,0,1,0,0,0,0,0,0,0,0,0.086,67656,78203,1,2017,4,19055,59.81
552,2017-01-27,2017-02-03,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"5102 OLD PEARSALL, San Antonio, 78242",4,2097121.0,13678237.0,1393.0,8,-1385.0,174.125,Very Late Response,0,0,0,1,0,0,0,0,0,0,0.078,66370,78242,1,2017,4,18500,65.21
553,2017-02-03,2017-02-10,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"2223 HOUSTON ST E, San Antonio, 78202",2,2140359.0,13702579.0,1386.0,8,-1378.0,173.250,Very Late Response,0,1,0,0,0,0,0,0,0,0,0.086,67656,78202,2,2017,5,19055,59.81
554,2017-02-03,2017-02-10,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"2531 PEREZ, San Antonio, 78207",1,2117463.0,13706003.0,1386.0,8,-1378.0,173.250,Very Late Response,1,0,0,0,0,0,0,0,0,0,0.148,68081,78207,2,2017,5,23967,26.00
555,2017-02-28,2017-03-07,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"8002 GRISSOM RD, San Antonio, 78251",6,2081307.0,13721187.0,1361.0,8,-1353.0,170.125,Very Late Response,0,0,0,0,0,1,0,0,0,0,0.124,80007,78251,2,2017,9,23437,38.44


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356804 entries, 551 to 399979
Data columns (total 36 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   open_date                 356804 non-null  datetime64[ns]
 1   due_date                  356804 non-null  datetime64[ns]
 2   closed_date               356804 non-null  datetime64[ns]
 3   is_late                   356804 non-null  object        
 4   dept                      356804 non-null  object        
 5   call_reason               356804 non-null  object        
 6   case_type                 356804 non-null  object        
 7   case_status               356804 non-null  object        
 8   source_id                 356804 non-null  object        
 9   address                   356804 non-null  object        
 10  council_district          356804 non-null  int64         
 11  longitude                 356804 non-null  float64       
 12  

In [ ]:
error = df[df.days_before_or_after_due == -1]

In [ ]:
error.level_of_delay.value_counts()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.level_of_delay.value_counts()

In [ ]:
df.is_late.value_counts()

In [ ]:
df.days_before_or_after_due.value_counts()

In [ ]:
on_time = df[df.level_of_delay == 'On Time Response']
very_late = df[df.level_of_delay == 'Very Late Response']
early = df[df.level_of_delay == 'Early Response']

In [ ]:
on_time.days_before_or_after_due.value_counts()

In [ ]:
very_late.days_before_or_after_due.value_counts()

In [ ]:
early.days_before_or_after_due.value_counts()

In [ ]:
df[df.days_before_or_after_due < 0]

In [5]:
def get_late(days_before_or_after_due):
    if days_before_or_after_due < 0:
        return 'late'
    else:
        return 'on time'
        

In [6]:
#get actual amount of late cases
df['late'] = df['days_before_or_after_due'].apply(get_late)

In [ ]:
#df.to_csv('final2_311.csv')

In [ ]:
# create new feature to show how long it took to resolve compared to resolution due date
#df['days_before_or_after_due'] = df.resolution_days_due - df.days_open

In [ ]:
import model as m

In [ ]:
df = m.model_df()

In [ ]:
  
import pandas as pd
import numpy as np
import wrangle

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn import preprocessing

#-----------------------------------------------------------------------------

# Model Prep

def dummy_dept(df):
    # dummy dept feature
    dummy_df =  pd.get_dummies(df['dept'])
    # Name the new columns
    dummy_df.columns = ['animal_care_services', 'code_enforcement_services', 
                        'customer_services', 'development_services', 
                        'metro_health', 'parks_and_rec',
                        'solid_waste_management', 'trans_and_cap_improvements', 
                        'unknown_dept']
    # add the dummies to the data frame
    df = pd.concat([df, dummy_df], axis=1)
    return df
#-----------------------------------------------------------------------------    
def dummy_call_reason(df):
    # dummy dept feature
    dummy_df =  pd.get_dummies(df['call_reason'])
    # Name the new columns
    dummy_df.columns = ['buildings', 'business', 'cleanup', 'code',
                        'customer_service', 'field', 'land',
                        'license', 'misc', 'storm', 'streets', 'trades', 
                        'traffic', 'waste']
    # add the dummies to the data frame
    df = pd.concat([df, dummy_df], axis=1)
    return df
#-----------------------------------------------------------------------------
def make_source_id_dummies(df):
    '''This function takes in the cleaned dataframe, makes dummy variables of the source id column, readds the names of the
    dummy columns and returns the concatenated dummy dataframe to the original dataframe.'''
    #make dummies
    dummy_df = pd.get_dummies(df['source_id'])
    #add back column names
    dummy_df.columns = ['web_portal', '311_mobile_app', 'constituent_call', 'internal_services_requests']
    # concatenate dummies to the cleaned data frame
    df = pd.concat([df, dummy_df], axis=1)
    return df

#-------------------------------
def keep_info(df):
    df.drop(df.columns.difference(['days_before_after_due','dept','call_reason', 'source_id', 'level_of_delay',
                                   'council_district', 'resolution_days_due', 'district_0', 'district_1', 'district_2',
                                   'district_3', 'district_4','district_5', 'district_6', 'district_7', 'district_8', 
                                   'district_9','district_10']), 1, inplace=True)
    return df

#--------------------------------
def model_df():
    '''This function reads in the clean 311 dataframe, applies all of the above functions to prepare it for modeling. 
    The function then returns a cleaned dataframe ready for modeling.'''
    df= wrangle.clean_311(wrangle.get_311_data())
    df= keep_info(df)
    df= dummy_dept(df)
    df= dummy_call_reason(df)
    df= make_source_id_dummies(df)

    return df
#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------

def split(df, stratify_by= 'level_of_delay'):
    """
    Crude train, validate, test split
    To stratify, send in a column name
    """
    if stratify_by == None:
        train, test = train_test_split(df, test_size=.2, random_state=319)
        train, validate = train_test_split(train, test_size=.3, random_state=319)
    else:
        train, test = train_test_split(df, test_size=.2, random_state=319, stratify=df[stratify_by])
        train, validate = train_test_split(train, test_size=.3, random_state=319, stratify=train[stratify_by])
    return train, validate, test


#------------------------------------
def separate_y(train, validate, test):
    '''
    This function will take the train, validate, and test dataframes and separate the target variable into its
    own panda series
    '''
    
    X_train = train.drop(columns=['level_of_delay'])
    y_train = train.level_of_delay
    X_validate = validate.drop(columns=['level_of_delay'])
    y_validate = validate.level_of_delay
    X_test = test.drop(columns=['level_of_delay'])
    y_test = test.level_of_delay
    return X_train, y_train, X_validate, y_validate, X_test, y_test
#------------------------------------
def scale_data(X_train, X_validate, X_test):
    '''
    This function will scale numeric data using Min Max transform after 
    it has already been split into train, validate, and test.
    '''
    
    
    obj_col = ['dept', 'call_reason','source_id']
    num_train = X_train.drop(columns = obj_col)
    num_validate = X_validate.drop(columns = obj_col)
    num_test = X_test.drop(columns = obj_col)
    
    
    # Make the thing
    scaler = preprocessing.MinMaxScaler()
    
   
    # we only .fit on the training data
    scaler.fit(num_train)
    train_scaled = scaler.transform(num_train)
    validate_scaled = scaler.transform(num_validate)
    test_scaled = scaler.transform(num_test)
    
    # turn the numpy arrays into dataframes
    train_scaled = pd.DataFrame(train_scaled, columns=num_train.columns)
    validate_scaled = pd.DataFrame(validate_scaled, columns=num_train.columns)
    test_scaled = pd.DataFrame(test_scaled, columns=num_train.columns)
    
    
    return train_scaled, validate_scaled, test_scaled

#------------------------------------

def split_separate_scale(df, stratify_by= 'level_of_delay'):
    '''
    This function will take in a dataframe
    separate the dataframe into train, validate, and test dataframes
    separate the target variable from train, validate and test
    then it will scale the numeric variables in train, validate, and test
    finally it will return all dataframes individually
    '''
    
    # split data into train, validate, test
    train, validate, test = split(df, stratify_by= 'level_of_delay')
    
     # seperate target variable
    X_train, y_train, X_validate, y_validate, X_test, y_test = separate_y(train, validate, test)
    
    
    # scale numeric variable
    train_scaled, validate_scaled, test_scaled = scale_data(X_train, X_validate, X_test)
    
    return train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test, train_scaled, validate_scaled, test_scaled

In [ ]:
train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test, train_scaled, validate_scaled, test_scaled = split_separate_scale(df, stratify_by= 'level_of_delay')

In [ ]:
train.head()

In [ ]:
train = train.drop(columns = ['dept', 'call_reason', 
        'source_id', 'council_district'])

In [ ]:
train = train.drop(columns = 'level_of_delay')

In [ ]:
df.head()

In [ ]:
df.late.value_counts()

### Level of Delay by District

https://public.tableau.com/app/profile/lori.segovia/viz/LevelofDelaybyDistrict/District1

In [ ]:
# 1 and 5 downtown/central
central = df[(df.district_1 == 1) | (df.district_5 == 1)]
# 8 and 9
north = df[(df.district_8 == 1) | (df.district_9 == 1)]
# 4 and 3
south = df[(df.district_4 == 1) | (df.district_3 == 1)]
# 10 and 2
east = df[(df.district_10 == 1) | (df.district_2 == 1)]
#6 and 7
west = df[(df.district_6 == 1) | (df.district_7 == 1)]

In [7]:
#make column for regions to explore in tableau

def get_regions(council_district):
    if council_district == 1:
        return 'Central'
    elif council_district == 2:
        return 'East'
    elif council_district == 3:
        return 'South'
    elif council_district == 4:
        return 'South'
    elif council_district == 5:
        return 'Central'
    elif council_district == 6:
        return 'West'
    elif council_district == 7:
        return 'West'
    elif council_district == 8:
        return 'North'
    elif council_district == 9:
        return 'North'
    else:
        return 'East'

In [8]:
#get actual amount of late cases
df['regions'] = df['council_district'].apply(get_regions)

In [9]:
df.head()

,open_date,due_date,closed_date,is_late,dept,call_reason,case_type,case_status,source_id,address,council_district,longitude,latitude,days_open,resolution_days_due,days_before_or_after_due,pct_time_of_used,level_of_delay,district_1,district_2,district_3,district_4,district_5,district_6,district_7,district_8,district_9,district_10,voter_turnout_2019,num_of_registered_voters,zipcode,open_month,open_year,open_week,per_capita_income,square_miles,late,regions
551,2017-01-27,2017-02-03,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"2407 WYOMING ST, San Antonio, 78203",2,2142962.0,13699885.0,1393.0,8,-1385.0,174.125,Very Late Response,0,1,0,0,0,0,0,0,0,0,0.086,67656,78203,1,2017,4,19055,59.81,late,East
552,2017-01-27,2017-02-03,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"5102 OLD PEARSALL, San Antonio, 78242",4,2097121.0,13678237.0,1393.0,8,-1385.0,174.125,Very Late Response,0,0,0,1,0,0,0,0,0,0,0.078,66370,78242,1,2017,4,18500,65.21,late,South
553,2017-02-03,2017-02-10,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"2223 HOUSTON ST E, San Antonio, 78202",2,2140359.0,13702579.0,1386.0,8,-1378.0,173.250,Very Late Response,0,1,0,0,0,0,0,0,0,0,0.086,67656,78202,2,2017,5,19055,59.81,late,East
554,2017-02-03,2017-02-10,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"2531 PEREZ, San Antonio, 78207",1,2117463.0,13706003.0,1386.0,8,-1378.0,173.250,Very Late Response,1,0,0,0,0,0,0,0,0,0,0.148,68081,78207,2,2017,5,23967,26.00,late,Central
555,2017-02-28,2017-03-07,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"8002 GRISSOM RD, San Antonio, 78251",6,2081307.0,13721187.0,1361.0,8,-1353.0,170.125,Very Late Response,0,0,0,0,0,1,0,0,0,0,0.124,80007,78251,2,2017,9,23437,38.44,late,West


In [10]:
#df.to_csv('newest_311.csv')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356804 entries, 551 to 399979
Data columns (total 38 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   open_date                 356804 non-null  datetime64[ns]
 1   due_date                  356804 non-null  datetime64[ns]
 2   closed_date               356804 non-null  datetime64[ns]
 3   is_late                   356804 non-null  object        
 4   dept                      356804 non-null  object        
 5   call_reason               356804 non-null  object        
 6   case_type                 356804 non-null  object        
 7   case_status               356804 non-null  object        
 8   source_id                 356804 non-null  object        
 9   address                   356804 non-null  object        
 10  council_district          356804 non-null  int64         
 11  longitude                 356804 non-null  float64       
 12  

In [12]:
df.per_capita_income.value_counts()

13836    50177
19055    47176
23967    46112
18281    45902
18500    34767
25263    33596
23437    30518
30240    29531
42559    19527
35475    19498
Name: per_capita_income, dtype: int64

In [15]:
compliments = df[df.case_type == 'Compliment']

In [17]:
compliments.shape

(10, 38)

In [18]:
compliments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 557 to 5758
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   open_date                 10 non-null     datetime64[ns]
 1   due_date                  10 non-null     datetime64[ns]
 2   closed_date               10 non-null     datetime64[ns]
 3   is_late                   10 non-null     object        
 4   dept                      10 non-null     object        
 5   call_reason               10 non-null     object        
 6   case_type                 10 non-null     object        
 7   case_status               10 non-null     object        
 8   source_id                 10 non-null     object        
 9   address                   10 non-null     object        
 10  council_district          10 non-null     int64         
 11  longitude                 10 non-null     float64       
 12  latitude            

In [19]:
compliments

,open_date,due_date,closed_date,is_late,dept,call_reason,case_type,case_status,source_id,address,council_district,longitude,latitude,days_open,resolution_days_due,days_before_or_after_due,pct_time_of_used,level_of_delay,district_1,district_2,district_3,district_4,district_5,district_6,district_7,district_8,district_9,district_10,voter_turnout_2019,num_of_registered_voters,zipcode,open_month,open_year,open_week,per_capita_income,square_miles,late,regions
557,2017-03-08,2017-03-09,2020-10-12,YES,Customer Service,customer_service,Compliment,Closed,Web Portal,"515 FRIO S, San Antonio, 78207",5,2126090.0,13701005.0,1315.0,2,-1313.0,657.500000,Very Late Response,0,0,0,0,1,0,0,0,0,0,0.085,61418,78207,3,2017,10,13836,22.24,late,Central
2320,2018-07-26,2018-08-06,2020-10-12,YES,Customer Service,customer_service,Compliment,Closed,Constituent Call,"15210 BENT MOSS, San Antonio, 78232",9,2144741.0,13758405.0,810.0,12,-798.0,67.500000,Very Late Response,0,0,0,0,0,0,0,0,1,0,0.185,99309,78232,7,2018,30,42559,48.71,late,North
2345,2018-08-02,2018-08-13,2020-10-12,YES,Customer Service,customer_service,Compliment,Closed,Constituent Call,"151 STOREYWOOD, San Antonio, 78213",1,2118504.0,13730803.0,803.0,12,-791.0,66.916667,Very Late Response,1,0,0,0,0,0,0,0,0,0,0.148,68081,78213,8,2018,31,23967,26.00,late,Central
2692,2018-10-31,2018-11-09,2020-10-12,YES,Customer Service,customer_service,Compliment,Closed,Constituent Call,"9402 FOREST OAK, San Antonio, 78217",10,2147080.0,13739031.0,713.0,10,-703.0,71.300000,Very Late Response,0,0,0,0,0,0,0,0,0,1,0.154,91378,78217,10,2018,44,30240,55.62,late,East
2833,2018-11-30,2018-12-11,2020-10-12,YES,Customer Service,customer_service,Compliment,Closed,Constituent Call,"816 DAKOTA ST, San Antonio, 78203",2,2137403.0,13699077.0,683.0,12,-671.0,56.916667,Very Late Response,0,1,0,0,0,0,0,0,0,0,0.086,67656,78203,11,2018,48,19055,59.81,late,East
3568,2019-03-21,2019-04-01,2020-10-12,YES,Customer Service,customer_service,Compliment,Closed,Constituent Call,"918 JOSEPHINE E, San Antonio, 78208",2,2136969.0,13709259.0,572.0,12,-560.0,47.666667,Very Late Response,0,1,0,0,0,0,0,0,0,0,0.086,67656,78208,3,2019,12,19055,59.81,late,East
3640,2019-03-28,2019-04-08,2020-10-12,YES,Customer Service,customer_service,Compliment,Closed,Constituent Call,"11721 PARLIAMENT, San Antonio, 78213",9,2123211.0,13746988.0,565.0,12,-553.0,47.083333,Very Late Response,0,0,0,0,0,0,0,0,1,0,0.185,99309,78213,3,2019,13,42559,48.71,late,North
3809,2019-04-15,2019-04-24,2020-10-12,YES,Customer Service,customer_service,Compliment,Closed,Constituent Call,"256 KENMAR, San Antonio, 78220",2,2156586.0,13701557.0,547.0,10,-537.0,54.700000,Very Late Response,0,1,0,0,0,0,0,0,0,0,0.086,67656,78220,4,2019,16,19055,59.81,late,East
4842,2019-07-19,2019-07-30,2020-10-12,YES,Customer Service,customer_service,Compliment,Closed,Constituent Call,"300 GROFF, San Antonio, 78237",5,2103568.0,13707838.0,452.0,12,-440.0,37.666667,Very Late Response,0,0,0,0,1,0,0,0,0,0,0.085,61418,78237,7,2019,29,13836,22.24,late,Central
5758,2019-09-24,2019-10-03,2020-10-12,YES,Customer Service,customer_service,Compliment,Closed,Constituent Call,"111 ARANSAS AVE, San Antonio, 78210",2,2135994.0,13695285.0,385.0,10,-375.0,38.500000,Very Late Response,0,1,0,0,0,0,0,0,0,0,0.086,67656,78210,9,2019,39,19055,59.81,late,East


In [ ]:
if df[df.case_type == 'Compliment']

In [ ]:
df.drop(['B', 'C'], axis=1)

In [ ]:
train = train.drop(columns = 'level_of_delay')

In [ ]:
#north = df[(df.council_district == 8) | (df.council_district == 9)]

In [ ]:
#north.head()

In [ ]:
north.council_district.value_counts()

In [ ]:
avg_days_north = north.days_before_or_after_due.mean()

In [ ]:
avg_days_south = south.days_before_or_after_due.mean()

In [ ]:
avg_days_east = east.days_before_or_after_due.mean()

In [ ]:
avg_days_west = west.days_before_or_after_due.mean()

In [ ]:
avg_days_central = central.days_before_or_after_due.mean()

In [ ]:
import scipy.stats as stats

In [ ]:
null = 'there is no difference in the means'
alt = 'there is a difference in the means'

In [ ]:
print(avg_days_north, 'north')
print(avg_days_south, 'south')
print(avg_days_east, 'east')
print(avg_days_west, 'west')
print(avg_days_central, 'central')

#### north statistical tests

In [ ]:
# north and south
t, p = stats.ttest_ind(north.days_before_or_after_due, south.days_before_or_after_due)
t, p

In [ ]:
# north and east
t, p = stats.ttest_ind(north.days_before_or_after_due, east.days_before_or_after_due)
t, p

In [ ]:
# north and west
t, p = stats.ttest_ind(north.days_before_or_after_due, west.days_before_or_after_due)
t, p

In [ ]:
# north and central
t, p = stats.ttest_ind(north.days_before_or_after_due, central.days_before_or_after_due)
t, p

#### south statistical tests

In [ ]:
# south and east
t, p = stats.ttest_ind(east.days_before_or_after_due, south.days_before_or_after_due)
t, p

In [ ]:
# south and west
t, p = stats.ttest_ind(west.days_before_or_after_due, south.days_before_or_after_due)
t, p

In [ ]:
# south and central
t, p = stats.ttest_ind(central.days_before_or_after_due, south.days_before_or_after_due)
t, p

#### east statistical tests

In [ ]:
# east and west
t, p = stats.ttest_ind(east.days_before_or_after_due, west.days_before_or_after_due)
t, p

In [ ]:
# east and central
t, p = stats.ttest_ind(east.days_before_or_after_due, central.days_before_or_after_due)
t, p

#### west statistical tests

In [ ]:
# west and central
t, p = stats.ttest_ind(central.days_before_or_after_due, west.days_before_or_after_due)
t, p

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(df.days_before_or_after_due)

In [22]:
df.head()

,open_date,due_date,closed_date,is_late,dept,call_reason,case_type,case_status,source_id,address,council_district,longitude,latitude,days_open,resolution_days_due,days_before_or_after_due,pct_time_of_used,level_of_delay,district_1,district_2,district_3,district_4,district_5,district_6,district_7,district_8,district_9,district_10,voter_turnout_2019,num_of_registered_voters,zipcode,open_month,open_year,open_week,per_capita_income,square_miles,late,regions
551,2017-01-27,2017-02-03,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"2407 WYOMING ST, San Antonio, 78203",2,2142962.0,13699885.0,1393.0,8,-1385.0,174.125,Very Late Response,0,1,0,0,0,0,0,0,0,0,0.086,67656,78203,1,2017,4,19055,59.81,late,East
552,2017-01-27,2017-02-03,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"5102 OLD PEARSALL, San Antonio, 78242",4,2097121.0,13678237.0,1393.0,8,-1385.0,174.125,Very Late Response,0,0,0,1,0,0,0,0,0,0,0.078,66370,78242,1,2017,4,18500,65.21,late,South
553,2017-02-03,2017-02-10,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"2223 HOUSTON ST E, San Antonio, 78202",2,2140359.0,13702579.0,1386.0,8,-1378.0,173.250,Very Late Response,0,1,0,0,0,0,0,0,0,0,0.086,67656,78202,2,2017,5,19055,59.81,late,East
554,2017-02-03,2017-02-10,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"2531 PEREZ, San Antonio, 78207",1,2117463.0,13706003.0,1386.0,8,-1378.0,173.250,Very Late Response,1,0,0,0,0,0,0,0,0,0,0.148,68081,78207,2,2017,5,23967,26.00,late,Central
555,2017-02-28,2017-03-07,2020-11-19,YES,Customer Service,customer_service,Complaint,Closed,Constituent Call,"8002 GRISSOM RD, San Antonio, 78251",6,2081307.0,13721187.0,1361.0,8,-1353.0,170.125,Very Late Response,0,0,0,0,0,1,0,0,0,0,0.124,80007,78251,2,2017,9,23437,38.44,late,West


In [24]:
df[df.address == '1278 W PYRON AVE, San Antonio, 78211']

,open_date,due_date,closed_date,is_late,dept,call_reason,case_type,case_status,source_id,address,council_district,longitude,latitude,days_open,resolution_days_due,days_before_or_after_due,pct_time_of_used,level_of_delay,district_1,district_2,district_3,district_4,district_5,district_6,district_7,district_8,district_9,district_10,voter_turnout_2019,num_of_registered_voters,zipcode,open_month,open_year,open_week,per_capita_income,square_miles,late,regions


In [ ]:
 df[df.case_type == 'Compliment']